In [6]:
# 1. Import Necessary Libraries:

import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import (
    ColumnDataSource,
    Select,
    CustomJS,
)
from bokeh.layouts import column

In [28]:
# 2. Read Data from Excel Files:

staff_df = pd.read_excel("staff.xlsx")
department_df = pd.read_excel("departments.xlsx")
types_df = pd.read_excel("types.xlsx")

# Merge staff and department data with specified suffixes
staff_df_merged = staff_df.merge(department_df, on="Job", suffixes=("_staff", "_department"))

# Rename conflicting column if necessary:
if "Department_staff" in staff_df_merged.columns:
    staff_df_merged = staff_df_merged.rename(columns={"Department_staff": "Staff_Department"})

# Combine department information if needed, handling potential missing values
department = staff_df_merged["Staff_Department"].fillna("") + " " + staff_df_merged["Department_department"].fillna("")

# Align types data and handle the resulting tuple
types_aligned_df1, _ = types_df.set_index("Project").align(staff_df_merged.set_index("Project"), join="inner")  # Access only the first DataFrame

# Create data source with consistent structure
source = ColumnDataSource(data=dict(
    project=[], department=[], job=[], month=[], type=[]  # Adjust based on required fields
))

# Fill data source with merged data
source.data = dict(
    project=staff_df_merged["Project"].tolist(),
    department=department.tolist(),  # Access combined department information
    job=staff_df_merged["Job"].tolist(),
    month=staff_df_merged["Month"].tolist(),
    type=types_aligned_df1["Type"].tolist(),  # Use aligned data from the first DataFrame
)


KeyError: 'Staff_Department'

In [24]:
# Print column names before merge
print("Columns in staff_df:", staff_df.columns)
print("Columns in department_df:", department_df.columns)

# Merge staff and department data with specified suffixes
staff_df_merged = staff_df.merge(department_df, on="Job", suffixes=("_staff", "_department"))

# Print column names after merge
print("Columns in staff_df_merged:", staff_df_merged.columns)

# Combine department information if needed
department = staff_df_merged["Department_staff"] + " " + staff_df_merged["Department_department"]

Columns in staff_df: Index(['Job', 'Project', 'Month'], dtype='object')
Columns in department_df: Index(['Job', 'Department'], dtype='object')
Columns in staff_df_merged: Index(['Job', 'Project', 'Month', 'Department'], dtype='object')


KeyError: 'Department_staff'

In [ ]:
# 3. Create Data Source for the Bokeh Plot:

source = ColumnDataSource(data=dict(project=[], department=[], # Add other relevant columns))

In [ ]:
# 4. Define Interactive Components:

project_dropdown = Select(title="Select Project:", value=None, options=list(staff_df['Project'].unique()))
department_dropdown = Select(title="Select Department:", value=None, options=list(department_df['Department'].unique()))


In [ ]:
#  5. Create Initial Deployment Plan Plot:

plot = figure(x_range=list(staff_df['Job'].unique()),  # Adjust as needed
              y_range=list(staff_df['Project'].unique()),  # Adjust as needed
              title="Deployment Plan")
# Add visual elements (circles, bars, etc.) based on your desired plot type


In [ ]:
# 6. Implement Callback to Update Plot Based on Selections:

callback = CustomJS(args=dict(source=source, project_dropdown=project_dropdown, department_dropdown=department_dropdown),
                    code="""
    // Filter data based on selected project and department
    const project = project_dropdown.value;
    const department = department_dropdown.value;
    const filtered_data = source.data;
    filtered_data['project'] = [...]; // Update project data based on filtering
    filtered_data['department'] = [...]; // Update department data based on filtering
    // ... (Update other data fields within the source dictionary)
    source.change.emit();
""")

project_dropdown.js_on_change('value', callback)
department_dropdown.js_on_change('value', callback)


In [ ]:
# 7. Arrange Layout and Display App:

layout = column(project_dropdown, department_dropdown, plot)
show(layout)
